In [18]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit.circuit.library import TwoLocal
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import StatevectorEstimator as Estimator
from scipy.optimize import minimize
from numpy.linalg import eigvalsh
import time
import imageio
import os

# Observable coefficients for 4 qubits (C₂ molecule)
observable = SparsePauliOp.from_list([
    ("IIXX", -1),
    ("IXIZ", 4.5),
    ("IZII", -92.6),
    ("YIII", 92.6),
    ("YZZI", -3.3),
    ("ZYZI", -3.3),
    ("YZIX", -500),
    ("IZIZ", 0.126),
    ("IIZY", -5.26),
    ("ZZZZ", 31.3)
])

def cost_func_vqe(params, ansatz, hamiltonian, estimator, energy_vals, step):
    """Return estimate of energy from estimator and store the value."""
    pub = (ansatz, hamiltonian, params)
    cost = estimator.run([pub]).result()[0].data.evs
    energy_vals[step] = cost
    return cost

# Define the reference circuit
reference_circuit = QuantumCircuit(4)
reference_circuit.x(0)  # Initial state for the first qubit

# Create the variational form (ansatz)
variational_form = TwoLocal(
    4,
    rotation_blocks=["rz", "ry","rx"],
    entanglement_blocks="cz",
    entanglement="full",
    reps=1,
)
raw_ansatz = reference_circuit.compose(variational_form)

# Initialize the estimator
estimator = Estimator()

# Initial parameters
x0 = np.ones(raw_ansatz.num_parameters)
num_iterations = 100

# List to store energy values
energy_vals = np.zeros(num_iterations)

# Minimization process using COBYLA
start_time = time.time()
result = minimize(
    cost_func_vqe, 
    x0, 
    args=(raw_ansatz, observable, estimator, energy_vals, 0), 
    method="COBYLA", 
    options={'maxiter': num_iterations, 'disp': True}
)
end_time = time.time()

execution_time = end_time - start_time
solution_eigenvalue = min(eigvalsh(observable.to_matrix()))

# Print the results
print(f"Number of iterations: {result.nfev}")
print(f"Time (s): {execution_time}")
print(f"Percent error: {abs((result.fun - solution_eigenvalue)/solution_eigenvalue):.2e}")

# Create GIF of energy minimization
filenames = []

for i in range(num_iterations):
    plt.figure()
    plt.plot(np.arange(i + 1), energy_vals[:i + 1], marker='o', color='blue')
    plt.title('Minimization of Energy During VQE Optimization for some Molecule')
    plt.xlabel('Iterations')
    plt.ylabel('Energy')
    plt.ylim(min(energy_vals) - 10, max(energy_vals) + 10)  # Adjust Y limits for clarity
    plt.grid()
    
    # Save each frame as a PNG file
    img_file = f"frame_{i}.png"
    plt.savefig(img_file)
    plt.close()  # Close the figure to avoid display
    filenames.append(img_file)

# Create GIF from images
with imageio.get_writer('vqe_energy_minimization-2.gif', mode='I', duration=0.1) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Clean up the frame images if desired
for filename in filenames:
    os.remove(filename)


Number of iterations: 100
Time (s): 0.6277961730957031
Percent error: 2.92e-01

   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =  100   F =-4.912148E+02    MAXCV = 0.000000E+00
   X = 2.464435E+00   9.769170E-01   8.922622E-01   6.199303E-02   3.952429E-01
       1.862646E-01   2.118607E+00   1.808425E+00   1.224528E+00  -3.931967E-02
       1.778476E+00   1.216578E+00   6.737917E-01   2.637809E-01   8.215444E-01
       1.077440E+00   1.352994E+00   2.354573E+00   8.320801E-02   6.684930E-01
       2.219083E+00   1.779054E+00  -2.016000E-01   4.992673E-01


/tmp/ipykernel_117186/4024888065.py:98: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [12]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit.circuit.library import TwoLocal
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import StatevectorEstimator as Estimator
from scipy.optimize import minimize
from numpy.linalg import eigvalsh
import time
import imageio
import os

# Observable coefficients for 4 qubits (C₂ molecule)
observable = SparsePauliOp.from_list([
    ("IIXX", 1),
    ("IXIZ", 4.5),
    ("IZII", -92.6),
    ("ZIII", 92.6),
    ("IZZI", -3.3),
    ("ZYZI", -3.3),
    ("ZZIX", -3.3),
    ("IZIZ", 0.1),
    ("IIZY", 5.26),
    ("ZZZZ", 31.3)
])

def cost_func_vqe(params, ansatz, hamiltonian, estimator, energy_vals, step):
    """Return estimate of energy from estimator and store the value."""
    pub = (ansatz, hamiltonian, params)
    cost = estimator.run([pub]).result()[0].data.evs
    energy_vals[step] = cost
    return cost

# Define the reference circuit
reference_circuit = QuantumCircuit(4)
reference_circuit.x(0)  # Initial state for the first qubit

# Create the variational form (ansatz)
variational_form = TwoLocal(
    4,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="full",
    reps=5,
)
raw_ansatz = reference_circuit.compose(variational_form)

# Initialize the estimator
estimator = Estimator()

# Initial parameters
x0 = np.ones(raw_ansatz.num_parameters)
num_iterations = 25

# List to store energy values
energy_vals = np.zeros(num_iterations)

# Minimization process using COBYLA
start_time = time.time()
result = minimize(
    cost_func_vqe, 
    x0, 
    args=(raw_ansatz, observable, estimator, energy_vals, 0), 
    method="COBYLA", 
    options={'maxiter': num_iterations, 'disp': True}
)
end_time = time.time()

execution_time = end_time - start_time
classical_energy = eigvalsh(observable.to_matrix())
classical_min_energy = np.min(classical_energy)

# Print results
print(f"Number of iterations: {result.nfev}")
print(f"Time (s): {execution_time}")
print(f"Percent error: {abs((result.fun - classical_min_energy) / classical_min_energy):.2e}")
print(f"Classical compute energy: {classical_energy}")
print(f"Minimum classical compute energy: {classical_min_energy}")

# Create GIF of energy minimization
filenames = []

for i in range(num_iterations):
    plt.figure()
    plt.plot(np.arange(i + 1), energy_vals[:i + 1], marker='o', color='blue', label='Quantum Energy')
    plt.axhline(y=classical_min_energy, color='red', linestyle='--', label='Classical Energy')
    
    # Adjust Y limits dynamically
    plt.ylim(min(min(energy_vals), classical_min_energy) - 10, 
             max(max(energy_vals), classical_min_energy) + 10)
    
    plt.title('Minimization of Energy During VQE Optimization for C₂ Molecule')
    plt.xlabel('Iterations')
    plt.ylabel('Energy (scaled)')
    plt.grid()
    plt.legend()
    
    # Save each frame as a PNG file
    img_file = f"frame_{i}.png"
    plt.savefig(img_file)
    plt.close()  # Close the figure to avoid display
    filenames.append(img_file)

# Create GIF from images
with imageio.get_writer('vqe_energy_minimization_with_classical.gif', mode='I', duration=0.1) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Clean up the frame images if desired
for filename in filenames:
    os.remove(filename)


Number of iterations: 25
Time (s): 0.2517518997192383
Percent error: 5.92e-01
Classical compute energy: [-220.5874714  -213.88550578 -157.07149062 -150.01646583  -35.57502298
  -35.18353417  -28.49797653  -28.12850938   28.31553454   28.68550578
   34.99668876   35.3874714   150.20331124  156.88446546  213.69797653
  220.77502298]
Minimum classical compute energy: -220.58747140404185

   Return from subroutine COBYLA because the MAXFUN limit has been reached.

   NFVALS =   25   F =-9.002984E+01    MAXCV = 0.000000E+00
   X = 1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00
       1.000000E+00   1.000000E+00   1.000000E+00   2.000000E+00   1.000000E+00
       1.000000E+00   1.000000E+00   1.000000E+00   2.000000E+00   2.000000E+00
       1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00
       2.000000E+00   2.000000E+00   2.000000E+00   2.000000E+00   1.000000E+00
       1.000000E+00   1.000000E+00   1.000000E+00   1.000000E+00   1.000000

/tmp/ipykernel_117186/3710561580.py:107: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [17]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit.circuit.library import TwoLocal
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import StatevectorEstimator as Estimator
from scipy.optimize import minimize
from numpy.linalg import eigvalsh
import time
import imageio
import os

# Observable coefficients for 4 qubits (C₂ molecule)
observable = SparsePauliOp.from_list([
    ("IIXX", 1),
    ("IXIZ", 4.5),
    ("IZII", -92.6),
    ("ZIII", 92.6),
    ("IZZI", -3.3),
    ("ZYZI", -3.3),
    ("ZZIX", -3.3),
    ("IZIZ", 0.1),
    ("IIZY", 5.26),
    ("ZZZZ", 31.3)
])

def cost_func_vqe(params, ansatz, hamiltonian, estimator, energy_vals, step):
    """Return estimate of energy from estimator and store the value."""
    pub = (ansatz, hamiltonian, params)
    cost = estimator.run([pub]).result()[0].data.evs
    energy_vals[step] = cost
    return cost

# Define the reference circuit
reference_circuit = QuantumCircuit(4)
reference_circuit.x(0)  # Initial state for the first qubit

# Create the variational form (ansatz)
variational_form = TwoLocal(
    4,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="full",
    reps=5,
)
raw_ansatz = reference_circuit.compose(variational_form)

# Initialize the estimator
estimator = Estimator()

# Initial parameters
x0 = np.ones(raw_ansatz.num_parameters)
num_iterations = 25

# List to store energy values
energy_vals = np.zeros(num_iterations)

# Minimization process using SLSQP
start_time = time.time()
result = minimize(
    cost_func_vqe, 
    x0, 
    args=(raw_ansatz, observable, estimator, energy_vals, 0), 
    method="SLSQP",  # Change to SLSQP
    options={'maxiter': num_iterations, 'disp': True}
)
end_time = time.time()

execution_time = end_time - start_time
solution_eigenvalue = min(eigvalsh(observable.to_matrix()))

# Classically computed energy (using exact diagonalization)
classical_energy = eigvalsh(observable.to_matrix())
classical_min_energy = np.min(classical_energy)

# Print the results
print(f"Number of iterations: {result.nfev}")
print(f"Time (s): {execution_time}")
print(f"Percent error: {abs((result.fun - classical_min_energy)/classical_min_energy):.2e}")
print(f"Classical compute energy: {classical_energy} ")
print(f"Minimum classical computer energy: {classical_min_energy}")

# Create GIF of energy minimization
filenames = []

for i in range(num_iterations):
    plt.figure()
    plt.plot(np.arange(i + 1), energy_vals[:i + 1], marker='o', color='blue', label='Quantum Energy')
    plt.axhline(y=classical_min_energy, color='red', linestyle='--', label='Classical Energy')
    plt.title('Minimization of Energy During VQE Optimization for C₂ Molecule')
    plt.xlabel('Iterations')
    plt.ylabel('Energy (scaled)')
    plt.ylim(min(energy_vals) - 10, max(energy_vals) + 10)  # Adjust Y limits for clarity
    plt.grid()
    plt.legend()
    
    # Save each frame as a PNG file
    img_file = f"frame_{i}.png"
    plt.savefig(img_file)
    plt.close()  # Close the figure to avoid display
    filenames.append(img_file)

# Create GIF from images
with imageio.get_writer('vqe_energy_minimization_with_classical(SLSQP).gif', mode='I', duration=0.1) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

# Clean up the frame images if desired
for filename in filenames:
    os.remove(filename)


Iteration limit reached    (Exit mode 9)
            Current function value: -217.11086501934903
            Iterations: 25
            Function evaluations: 1306
            Gradient evaluations: 25
Number of iterations: 1306
Time (s): 12.79030156135559
Percent error: 1.58e-02
Classical compute energy: [-220.5874714  -213.88550578 -157.07149062 -150.01646583  -35.57502298
  -35.18353417  -28.49797653  -28.12850938   28.31553454   28.68550578
   34.99668876   35.3874714   150.20331124  156.88446546  213.69797653
  220.77502298] 
Minimum classical computer energy: -220.58747140404185


/tmp/ipykernel_117186/2883571264.py:106: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
